In [7]:
import pyodbc as pyodbc
import pandas_datareader as datareader
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline
import yfinance as yf
from config import ROOT_DIR

#### Connect to sql server db and create stocks table

In [12]:
DRIVER = 'SQL Server'
SERVER_NAME = 'LAPTOP-S1EISQB0'
DB_NAME = 'newdb'

In [13]:
conn_str = (f"Driver={DRIVER};Server={SERVER_NAME};Database={DB_NAME};Trusted_Connection=yes;")
conn = pyodbc.connect(conn_str)
conn.autocommit = True 
cursor = conn.cursor()

In [14]:
query = """
use newdb;
create table stocks (
Date date not null PRIMARY KEY,
Open_Price numeric(9, 6) not null,
Close_Price numeric(9, 6) not null,
High_Price numeric(9, 6) not null,
Low_Price numeric(9, 6) not null,
Adj_Close numeric(9, 6) not null,
Volume numeric(9, 6) not null
);
"""""

In [15]:
cursor.execute(query)

In [43]:
yf.pdr_override()
startdate = datetime(2006,1,1)
enddate = datetime(2016,1,1)

In [46]:
data = pdr.get_data_yahoo(["BAC"], start=startdate, end=enddate)

[*********************100%***********************]  1 of 1 completed


In [49]:
data.reset_index(inplace=True)

In [60]:
data.to_csv(rf"{ROOT_DIR}\data\stocks.csv", index=False)

In [61]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-01-03,46.919998,47.180000,46.150002,47.080002,33.170311,16296700
1,2006-01-04,47.000000,47.240002,46.450001,46.580002,32.818035,17757900
2,2006-01-05,46.580002,46.830002,46.320000,46.639999,32.860313,14970700
3,2006-01-06,46.799999,46.910000,46.349998,46.570000,32.810986,12599800
4,2006-01-09,46.720001,46.970001,46.360001,46.599998,32.832119,15619400
...,...,...,...,...,...,...,...
2512,2015-12-24,17.320000,17.379999,17.219999,17.270000,15.006725,29369400
2513,2015-12-28,17.219999,17.230000,16.980000,17.129999,14.885072,41777500
2514,2015-12-29,17.250000,17.350000,17.160000,17.280001,15.015417,45670400
2515,2015-12-30,17.200001,17.240000,17.040001,17.049999,14.815555,35066400


In [64]:
data["Date"].nunique()

2517

In [68]:
query = """
use newdb;
create table stocks (
Date date not null PRIMARY KEY,
Open_Price int not null,
Close_Price int not null,
High_Price int not null,
Low_Price int not null,
Adj_Close int not null,
Volume int not null
);
"""""